In [1]:
import pyspark.sql.functions as psf
from pyspark.ml.feature import CountVectorizer
import pandas as pd
import numpy as np
test = spark.read.csv('FileStore/tables/test.csv', header = True)
train = spark.read.csv('FileStore/tables/train.csv', header = True)

In [2]:
#1) data cleaning
#function for finding number of NAs in each column, to see if interpolation, data dropping, needed | add: min, max, mean, presence of outliers etc. to see if if/where cleaning necessary
def clean_info(df):
  n_col = len(df.columns)
  n_row = df.count()
  #initializing results dataframe
  data = {'Name':df.columns, 'Nas':np.repeat(0, n_col), 'Percent':np.repeat(0,n_col)} 
  result = pd.DataFrame(data) 
  #loop through each column and get number of NAs
  for i in range(0, n_col):
    #name
    result.iloc[i,0] = df.columns[i]
    #NAs
    result.iloc[i,1] = n_row - df.select(df.columns[i]).drop().count()
    #perc
    result.iloc[i,2] = result.iloc[i,1] / n_row
  return result

#this data is already cleaned, so no NAs, erroneous data, outliers, etc.
clean_info(train)

In [3]:
#2) feature engineering
#function to create features, in order to be able to apply to test set later on
def create_features(df):
  #change pickup datetime to datetime from string, day of week
  df.createOrReplaceTempView('data')
  to_datetime = "SELECT *, TO_TIMESTAMP(SUBSTRING(pickup_datetime, 1, 19)) AS pickup_datetime_new, DATE_FORMAT(TO_TIMESTAMP(SUBSTRING(pickup_datetime, 1, 19)), 'EEEE') AS dow FROM data"
  df = spark.sql(to_datetime)
  
  #create columns for year, month, day, hour, minute
  df = df.withColumn('year', psf.year(df.pickup_datetime_new))
  df = df.withColumn('month', psf.month(df.pickup_datetime_new))
  df = df.withColumn('day', psf.dayofmonth(df.pickup_datetime_new))
  df = df.withColumn('hour', psf.hour(df.pickup_datetime_new))
  df = df.withColumn('minute', psf.minute(df.pickup_datetime_new))
  
  #converting day of week to one hot encoding
  df = df.withColumn('dow_array', psf.split(psf.col('dow'),' '))
  dowVectorizer = CountVectorizer(inputCol='dow_array', outputCol='dow_one_hot', vocabSize=7, minDF=1.0)
  dowVectorizer_model = dowVectorizer.fit(df)
  df = dowVectorizer_model.transform(df)
  
  #holidays, static ones, could encode year specific or use library
  holidays = [(1,1),(7,4),(12,25)]
  df = df.withColumn('holiday', psf.lit(0))
  for i in range(0, len(holidays)):
    df = df.withColumn('holiday', ((df.day == holidays[i][1]) & (df.month == holidays[i][0])) | (df.holiday == True))
  
  #night time surcharge beween 20 and 6
  df = df.withColumn('night', (df.hour >= 20) | (df.hour <= 6))
  
  #dropping unnecessary columns
  drops = ['key', 'pickup_datetime', 'pickup_datetime_new', 'dow', 'dow_array']
  for i in drops:
    df = df.drop(i)
  
  return df

#creating the engineered training data
training_data = create_features(train)

In [4]:
#3) create pipeline
#4) train test split, create model
#5) cross validation and grid search parameters
#6) select the best model
#7) predict

+-----------+----------------+---------------+-----------------+-----------------+---------------+----+-----+---+----+------+-------------+-------+-----+
fare_amount|pickup_longitude|pickup_latitude|dropoff_longitude| dropoff_latitude|passenger_count|year|month|day|hour|minute| dow_one_hot|holiday|night|
+-----------+----------------+---------------+-----------------+-----------------+---------------+----+-----+---+----+------+-------------+-------+-----+
 4.5| -73.844311| 40.721319| -73.84161|40.71227800000001| 1|2009| 6| 15| 17| 26|(7,[6],[1.0])| false|false|
 16.9| -74.016048| 40.711303| -73.979268| 40.782004| 1|2010| 1| 5| 16| 52|(7,[4],[1.0])| false|false|
 5.7| -73.982738| 40.76127| -73.991242| 40.750562| 2|2011| 8| 18| 0| 35|(7,[2],[1.0])| false| true|
+-----------+----------------+---------------+-----------------+-----------------+---------------+----+-----+---+----+------+-------------+-------+-----+
only showing top 3 rows